In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix

# Load the dataset
data = pd.read_csv('/kaggle/input/fashion-product-images-dataset/fashion-dataset/styles.csv')

# Simulate user swipe data
# Assuming we have user_ids and item_ids for swipes (1 for right swipe, 0 for left swipe)
# For now, let's generate dummy swipe data (replace with actual swipe data)
np.random.seed(42)
user_ids = np.random.randint(1, 100, size=len(data))  # 100 users for simulation
swipes = np.random.randint(0, 2, size=len(data))  # Random 0/1 for left/right swipe

# Add user_id and swipe data to the dataset
data['user_id'] = user_ids
data['swipe'] = swipes

# Filter items that were swiped right
liked_items = data[data['swipe'] == 1]

# Prepare the user-item interaction matrix
user_item_matrix = coo_matrix((liked_items['swipe'], (liked_items['user_id'], liked_items['id'])))

# Split the data into training and testing sets
train_data, test_data = train_test_split(liked_items, test_size=0.2, random_state=42)


In [ ]:
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

# Build the ALS model
als_model = AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

# Train the ALS model on the user-item interaction matrix
als_model.fit(user_item_matrix.T)  # Transpose because ALS expects items x users

# Now, we can generate predictions for users
user_id = 1  # Example user ID
recommendations = als_model.recommend(user_id, user_item_matrix.T, N=10)

# Print recommended item IDs
print("Recommended item IDs for user:", [rec[0] for rec in recommendations])


In [ ]:
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

# Build the ALS model
als_model = AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

# Train the ALS model on the user-item interaction matrix
als_model.fit(user_item_matrix.T)  # Transpose because ALS expects items x users

# Now, we can generate predictions for users
user_id = 1  # Example user ID
recommendations = als_model.recommend(user_id, user_item_matrix.T, N=10)

# Print recommended item IDs
print("Recommended item IDs for user:", [rec[0] for rec in recommendations])

In [ ]:
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

# Build the ALS model
als_model = AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

# Train the ALS model on the user-item interaction matrix
als_model.fit(user_item_matrix.T)  # Transpose because ALS expects items x users

# Now, we can generate predictions for users
user_id = 1  # Example user ID
recommendations = als_model.recommend(user_id, user_item_matrix.T, N=10)

# Print recommended item IDs
print("Recommended item IDs for user:", [rec[0] for rec in recommendations])

In [ ]:
# Evaluate using Mean Average Precision at K (MAP@K)
mapk_score = mean_average_precision_at_k(als_model, user_item_matrix.T, test_data['user_id'].values, K=10)
print(f"MAP@10: {mapk_score}")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# Encode categorical features into numerical values
label_encoders = {}
categorical_features = ['masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage']

for feature in categorical_features:
    label_encoders[feature] = LabelEncoder()
    data[feature] = label_encoders[feature].fit_transform(data[feature])

# Function to recommend complementary items (cross-sell) based on an item
def recommend_complementary_items(item_id, data, n_recommendations=5):
    # Find the selected item
    selected_item = data[data['id'] == item_id]
    
    # Get the features of the selected item
    selected_item_features = selected_item[categorical_features].values
    
    # Compute cosine similarity between the selected item and all other items
    similarities = cosine_similarity(selected_item_features, data[categorical_features].values)
    
    # Get the top N similar items (excluding the selected item itself)
    similar_items_indices = similarities.argsort()[0][-n_recommendations-1:-1]
    
    # Return the top N recommended complementary items
    return data.iloc[similar_items_indices]

# Example: Recommend complementary items for a shirt (item ID 15970)
complementary_items = recommend_complementary_items(15970, data)
print("Complementary items:", complementary_items[['productDisplayName', 'articleType', 'baseColour']])

In [ ]:
def get_personalized_recommendations(user_id, als_model, user_item_matrix, data, n_recommendations=5):
    # Step 1: Collaborative Filtering to get personalized item recommendations
    als_recommendations = als_model.recommend(user_id, user_item_matrix.T, N=n_recommendations)
    
    # Step 2: For each recommended item, find complementary items using cross-sell
    all_recommendations = []
    for rec in als_recommendations:
        item_id = rec[0]
        complementary_items = recommend_complementary_items(item_id, data)
        all_recommendations.append((item_id, complementary_items[['productDisplayName', 'articleType', 'baseColour']]))
    
    return all_recommendations

# Example: Get personalized recommendations and complementary items for user 1
personalized_recommendations = get_personalized_recommendations(1, als_model, user_item_matrix, data)
for rec in personalized_recommendations:
    print("Recommended Item:", rec[0])
    print("Complementary Items:")
    print(rec[1])